In [ ]:
train_dir = "G:/Gallblader Diseases Dataset/train"
test_dir = "G:/Gallblader Diseases Dataset/test"
val_dir = "G:/Gallblader Diseases Dataset/val"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

input_shape=(64,64)

datagen_train = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)
datagen_val=ImageDataGenerator(rescale=1./255)
batch_size = 64


generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)

generator_val = datagen_val.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

In [ ]:
INPUT_SHAPE=(64,64,3)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import tensorflow as tf
model=Sequential()
vgg16=VGG16(include_top=False,input_shape=INPUT_SHAPE, weights=None)
model.add(vgg16)
model.add(Flatten())
model.add(Dense(9,activation="Softmax"))
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'], optimizer=optimizer)

In [ ]:
epochs = 50
steps_per_epoch = generator_train.n/batch_size
steps_val=generator_val.n/batch_size

In [ ]:
acc=float('-inf')
loss=float('inf')
model_acc=model
# model_loss=model
history_final={'loss': [],
 'categorical_accuracy': [],
 'val_loss': [],
 'val_categorical_accuracy': []}

for i in range(100):
    print(f'Epoch: {i+1}/{100}')
    history = model.fit_generator(generator=generator_train,
                                        epochs=1,
                                        steps_per_epoch=steps_per_epoch,
                                        validation_data=generator_val,
                                        validation_steps=steps_val)

    history_final['loss']+=history.history['loss']
    history_final['categorical_accuracy']+=history.history['categorical_accuracy']
    history_final['val_loss']+=history.history['val_loss']
    history_final['val_categorical_accuracy']+=history.history['val_categorical_accuracy']

    # if history.history['val_loss'][0]<loss:
    #     model_loss=model
    if history.history['val_categorical_accuracy'][0]>acc:
        model_acc=model

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history_final['categorical_accuracy'])
plt.plot(history_final['val_categorical_accuracy'])
# plt.title('ResINN Training and Validation Loss')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['val', 'train'], loc = 'lower right')
plt.show()

plt.plot(history_final['loss'])
plt.plot(history_final['val_loss'])
# plt.title('ResINN Training and Validation Accuracy')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['val', 'train'], loc = 'lower right')
plt.show()

In [ ]:
generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

In [ ]:
import numpy as np
predict = model_acc.predict(generator_test)
predict=np.argmax(predict,axis=1)

In [ ]:
from sklearn.metrics import classification_report
import pandas as pd
import seaborn as sns


clf_report_df = pd.DataFrame(classification_report(generator_test.classes, predict, target_names=['Gallstones', 'Abdomen and retroperitoneum', 'cholecystitis', 'Membranous and gangrenous cholecystitis', 'Perforation', 'Polyps and cholesterol crystals', 'Adenomyomatosis', 'Carcinoma', 'Various causes of gallbladder wall thickening'], output_dict=True, digits=4)).T

sns.heatmap(clf_report_df.iloc[:-1, :-1], annot=True, fmt=".4f", cmap=sns.cubehelix_palette(as_cmap=True))

In [ ]:
from sklearn.metrics import confusion_matrix
array = confusion_matrix(generator_test.classes, predict)
df_cm = pd.DataFrame(array, index = ['Gallstones', 'Abdomen and retroperitoneum', 'cholecystitis', 'Membranous and gangrenous cholecystitis', 'Perforation', 'Polyps and cholesterol crystals', 'Adenomyomatosis', 'Carcinoma', 'Various causes of gallbladder wall thickening'],
                  columns = ['Gallstones', 'Abdomen and retroperitoneum', 'cholecystitis', 'Membranous and gangrenous cholecystitis', 'Perforation', 'Polyps and cholesterol crystals', 'Adenomyomatosis', 'Carcinoma', 'Various causes of gallbladder wall thickening'])
plt.figure(figsize = (6,5))
cm_plot = sns.heatmap(df_cm, annot=True, cmap='GnBu', fmt='g')
cm_plot.set_xlabel('Predicted Labels')
cm_plot.set_ylabel('True Labels')
cm_plot.set_title('Confusion Matrix From ResNet', size=12)